In [1]:
import sys
sys.path.append('../')

In [2]:
%load_ext autoreload
%autoreload 2

# RAG

In [3]:
import os
import torch
from langchain.document_loaders import PyMuPDFLoader
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
device="cuda" if torch.cuda.is_available() else "cpu"
print(device)

cpu


In [ ]:
from src.config import PATH_PROJECT
CHROMA_PERSIST_DIR = os.path.join(PATH_PROJECT, "data/chroma_db/")

if not os.path.exists(CHROMA_PERSIST_DIR) or not os.listdir(CHROMA_PERSIST_DIR):
    print("🆕 Création de la base Chroma et indexation des documents...")

    # Charger les fichiers PDF
    data_dir = os.path.join(PATH_PROJECT, "data/")
    documents = []

    for file in os.listdir(data_dir):
        if file.endswith(".pdf"):
            pdf_path = os.path.join(data_dir, file)
            pdf_loader = PyMuPDFLoader(pdf_path)
            documents.extend(pdf_loader.load())

    # Diviser les documents en chunks
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    docs = text_splitter.split_documents(documents)

    embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2",
                                       model_kwargs={'device': device}, encode_kwargs={'device': device})

    # Initialiser Chroma avec les embeddings 
    vectorstore = Chroma.from_documents(docs, embedding_model, persist_directory=CHROMA_PERSIST_DIR)
    vectorstore.persist()
    print("Base Chroma sauvegardée avec succès")

else:
    print("Chargement de la base Chroma existante...")
    embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2",
                                       model_kwargs={'device': device}, encode_kwargs={'device': device})
    vectorstore = Chroma(persist_directory=CHROMA_PERSIST_DIR, embedding_function=embedding_model)

print("Base Chroma prête à être utilisée")

Chargement de la base Chroma existante...
Base Chroma prête à être utilisée


In [6]:
query = "What are the symptoms of depression ?"
retrieved_docs = vectorstore.similarity_search(query, k=3)

for doc in retrieved_docs:
    print(doc.page_content)  # Afficher les documents les plus pertinents

259
Mood disorders
•	 Depressive symptoms include depressed mood or markedly diminished interest or 
pleasure in activities.
•	 There are multiple anxiety symptoms, which may include feeling nervous, anxious or on 
edge, not being able to control worrying thoughts, fear that something awful will happen, 
having trouble relaxing, muscle tension or sympathetic autonomic symptoms.
•	 Neither the depressive nor the anxiety symptoms – considered separately – are sufficiently
guilt, anger, denial, blame, difficulty accepting the death, feeling one has lost a part of one’s self, 
inability to experience positive mood, emotional numbness, difficulty in engaging with social or 
other activities). Some common symptoms of prolonged grief disorder are similar to those observed 
in a depressive episode (e.g. sadness, loss of interest in activities, social withdrawal, feelings of guilt,
•	 The symptoms include:
•	 at least one affective symptom such as mood lability, irritability, depressed mood or 

In [ ]:
import ollama

context = "\n\n".join([doc.page_content for doc in retrieved_docs])
prompt = f"Contexte:\n{context}\n\nQuestion: {query}\nRéponse:"

response = ollama.chat(model="mistral", messages=[{"role": "user", "content": prompt}])
print(response['message'].content)

 Depression symptoms include depressed mood or markedly diminished interest or pleasure in activities, feelings of guilt or worthlessness, fatigue or loss of energy, difficulty concentrating, changes in appetite or sleep, and recurrent thoughts of death or suicide. Additionally, there may be somatic or cognitive symptoms such as lethargy, joint pain, overeating, hypersomnia, breast tenderness, swelling of extremities, concentration difficulties or forgetfulness. It's important to note that these symptoms can vary in severity and may not always be present for every individual experiencing depression. If you or someone else is struggling with these symptoms, it's essential to seek help from a healthcare professional.


# LLM only approach

In [3]:
from src.inference import get_answer

In [ ]:
get_answer(patient_id=10, model='mistral')

' The patient meets the criteria for SEED (Single Episodic Episode Depression). They have a history of one depressive episode and no history of manic, mixed or hypomanic episodes, indicating that they do not have Bipolar Disorder Type 1 or 2.'

In [20]:
get_answer(patient_id=1, model='deepseek-r1:8b')

"<think>\nOkay, so I need to figure out the diagnosis for this patient based on their observed symptoms and history. Let me start by looking at what's given.\n\nThe Observed symptoms are:\n- Depressed mood for 1.5 weeks.\n- Reduced concentration for 1.2 weeks.\n- Reduced energy for 0.8 weeks.\n- Increased talkativeness for 0.6 weeks.\n\nThe History shows that depressive and hypomanic episodes have each happened once.\n\nFirst, I should recall the diagnostic criteria for each disorder mentioned: BPD1, BPD2, SEED, and RDD.\n\nBPD1 requires a history of at least one manic or mixed episode. Here, the patient has a hypomanic episode in their history but no manic or mixed episodes observed now. Since they don't have a current manic or mixed episode, does that disqualify them? I think so because the history needs to include at least one manic or mixed, and if this is the only hypomanic, it might not meet BPD1.\n\nBPD2 requires a history of at least one hypomanic and one depressive episode. Th

In [ ]:
for i in range(2, 11):
    print(f"patient {i}", get_answer(patient_id=i, model='deepseek-r1:8b'))